# Replaying Old Data

- Work under a separate branch with git (i.e. git checkout -b my-new-branch)

- Make a new folder in Projects (for your specific purposes)

- Copy this ipynb file that folder

- The data is stored on L-Drive

- the main gist of re-playing existing data is:
    - that your first load everything into a big matrix with mne
    - then initialize a specific **amp** (i.e., the "replayamp")
    - then do exactly the same as normal, with a while True loop with calls to amp.get_data()
    

In [1]:
# import all the needed stuff:

import time
import sys
import numpy as np 
import matplotlib
import matplotlib.pyplot as plt

import easygui  # popup windows with buttons made easy
import mne  # EEGLAB for python
from IPython.display import clear_output  # to clear the cell output during while loop
import re  # regular expressions
import pickle  # to save/load data
import dynarray  # a growing numpy array

import logging
logging.basicConfig(level=logging.ERROR)

sys.path.append("../../mushu")  # driver for the amps
sys.path.append("../../mushu/libmushu")
import libmushu

sys.path.append("../../nftools")  # handy stuff needed for NF
from nftools.loopcontrol import LoopState
from nftools.analysis import convert_alld_allm_to_mne
from nftools.analysis import select_part_from_mne_dataset
from nftools.analysis import plot_compare_two_spectra


sys.path.append("../../wyrm")  # real-time data analysis
from wyrm.types import RingBuffer
from wyrm.types import BlockBuffer
from wyrm import io
from wyrm import processing as proc

import scipy
from scipy import signal

from collections import deque  # a FILO list useful for plotting!

### Thomas import ###
from numpy.linalg import multi_dot
from sklearn.decomposition import FastICA
### Thomas import end ###


In [4]:
# this is the replay notebook - so select a file for playback - this is for brainvision files.

#fn=easygui.fileopenbox(default='/media/ldrive/Lab_MichaelB/Johan/nf/rawdata/*.vhdr')
fn=easygui.fileopenbox(default='/media/ldrive/Lab_MichaelB/Johan/nf/rawdata/*.vhdr')
print(fn)

raw_fromfile = mne.io.read_raw_brainvision(fn)
montage=mne.channels.read_montage('standard_1005', ch_names=raw_fromfile.ch_names)  # always use MNE definitions
raw_fromfile.set_montage(montage)


### Thomas modified
train_length = raw_fromfile.times.shape[0] # length of data to train ICA on for eye blink removal (usually 30)
raw_fromfile.load_data()
raw_fromfile.set_eeg_reference('average', projection=True)
raw_fromfile.apply_proj()
raw_fromfile.filter(1., 35., n_jobs=1, fir_design='firwin')

raw_blinks = raw_fromfile.copy()
raw_blinks.load_data()
#raw_blinks.crop(0,train_length)
#raw_blinks.filter(1, None)

print('Raw_FromFile shape before: {0}'.format(raw_fromfile.get_data().shape))

%matplotlib qt5

ica = mne.preprocessing.ICA(method="infomax", random_state=1)
ica.fit(raw_blinks)
ica.plot_sources(inst=raw_blinks)
ica.plot_components(inst=raw_blinks)
ica_comps = ica.get_sources(inst=raw_blinks).get_data()

blks = raw_blinks.get_data().T
pca_comps = ica.pca_components_
M = ica.mixing_matrix_
M_inv = np.linalg.inv(M)
S = np.identity(M.shape[0])
plt.figure(); plt.plot(blks[:,0]); plt.show()
print([M.shape, M_inv.shape, S.shape, blks.shape])
print('Raw shape: {0}'.format(raw_blinks.get_data().shape))
print('Raw_FromFile shape after: {0}'.format(raw_fromfile.get_data().shape))
### Thomas modified end ###

L:\Lab_MichaelB\Johan\nf\rawdata\CH005\eeg\CHall_S05_2018-10-17_15-55-38.vhdr
Extracting parameters from L:\Lab_MichaelB\Johan\nf\rawdata\CH005\eeg\CHall_S05_2018-10-17_15-55-38.vhdr...
Setting channel info structure...
Currently, 1 trigger(s) will be dropped, such as [Impedance]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
The following EEG sensors did not have a position specified in the selected montage: ['EOG']. Their position has been left untouched.
Reading 0 ... 1169155  =      0.000 ...   584.577 secs...


<ipython-input-4-dd393dc0a51a>:7: RuntimeWarning: Currently, 1 trigger(s) will be dropped, such as [Impedance]. Consider using ``event_id`` to parse triggers that do not follow the 'S###' pattern.
  raw_fromfile = mne.io.read_raw_brainvision(fn)
<ipython-input-4-dd393dc0a51a>:9: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG']. Their position has been left untouched.
  raw_fromfile.set_montage(montage)


Adding average EEG reference projection.
1 projection items deactivated
Average reference projection was added, but has not been applied yet. Use the apply_proj method to apply it.
Created an SSP operator (subspace dimension = 1)
1 projection items activated
SSP projectors applied...
Setting up band-pass filter from 1 - 35 Hz
l_trans_bandwidth chosen to be 1.0 Hz
h_trans_bandwidth chosen to be 8.8 Hz
Filter length of 6601 samples (3.300 sec) selected
Raw_FromFile shape before: (65, 1169156)
Fitting ICA to data using 64 channels (please be patient, this may take a while)
Inferring max_pca_components from picks
Using all PCA components: 64
 


C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\mne\preprocessing\infomax_.py:192: RuntimeWarning: overflow encountered in exp
  y = 1.0 / (1.0 + np.exp(-u))


Fitting ICA took 425.9s.
[(64, 64), (64, 64), (64, 64), (1169156, 65)]
Raw shape: (65, 1169156)
Raw_FromFile shape after: (65, 1169156)


In [5]:
S[[0,1],:] = 0
eye_comp = ica_comps[0,:]

In [6]:
# properties of the recording, and what we're doing here:
fs = raw_fromfile.info['sfreq']
nbchan = raw_fromfile.info['nchan']-1

# for updating x-axis
data_l = raw_fromfile.get_data().shape[1]
data_length = np.linspace(0,data_l,num=data_l)

updateTime = 0.1  # run some kind of calculation every X seconds
buffSize = 1.0  # run calculation on last X seconds of data

In [7]:
%matplotlib qt5  
plt.ion()  # enable widget plots & interactive plots

time_in_plot=6
sy1=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list
sy2=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))  # for plotting - the FILO list
sy3=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))

channel_to_plot=0
sx=deque(np.zeros(round(fs * time_in_plot)), round(fs * time_in_plot))

featuresy1 = deque(np.zeros(round(1/updateTime * time_in_plot)), round(1/updateTime * time_in_plot))
featuresx = np.linspace(0, time_in_plot, round(1/updateTime * time_in_plot))

In [8]:
# real-time data filtering -- high-pass filter (of 1.0 Hz)

f_low = 1.0
# f_high = 15.0
butter_ord = 3
lenchannels = 64

#rt_b, rt_a = signal.butter(butter_ord, [f_low / fn, f_high / fn], btype='band')
rt_b_hp, rt_a_hp = signal.butter(butter_ord, 2*f_low/fs, btype='high', analog=False)  # a digital high-pass filter
rt_zi_hp = proc.lfilter_zi(rt_b_hp, rt_a_hp, lenchannels)


In [9]:
# real-time data filtering -- band-pass filter (of 12.0 - 15.0 Hz)

f_low = 12.0
f_high = 15.0
butter_ord = 3
lenchannels = 64

#rt_b, rt_a = signal.butter(butter_ord, [f_low / fn, f_high / fn], btype='band')
rt_b_bp, rt_a_bp = signal.butter(butter_ord, [2*f_low/fs, 2*f_high/fs], btype='band', analog=False)  # a digital high-pass filter
rt_zi_bp = proc.lfilter_zi(rt_b_bp, rt_a_bp, lenchannels)

In [10]:
# prepare data for replay (warning: need probably a lot of memory)

mul_factor = 1.0
if 1e-6 in [raw_fromfile.info['chs'][0]['cal'], raw_fromfile.info['chs'][0]['range']]:
    mul_factor = 1.0 / 1e-6

seed_d=raw_fromfile[:-1,:][0] * mul_factor  # scale the data to seed (so no 1e-6 stuff in the replayed data)
seed_d=np.array(seed_d.transpose())
seed_ch=raw_fromfile.ch_names[0:-1]
seed_fs=raw_fromfile.info['sfreq']

# prepare for replay; markers:
seed_mdata=np.transpose(raw_fromfile[-1,:][0])
seed_m=[[i / raw_fromfile.info['sfreq'] * 1000, int(m[0])] for i, m in enumerate(seed_mdata) if m > 0] 

In [11]:
amp = libmushu.get_amp('replayamp')
amp.configure(seed_d, seed_m, seed_ch, seed_fs, realtime=True, blocksize_samples=100)

In [12]:
alld=dynarray.DynamicArray((None, len(amp.get_channels())))     # data
allm=[]     # markers
sfreq = amp.get_sampling_frequency()  # sampling frequency
ch_names=amp.get_channels()  # channel names

markTime=time.time()

rb = RingBuffer(buffSize * 1000)  # the buffer containing the last X seconds of data - declared in MILISECONDS
totalTime = seed_d.shape[0]/raw_fromfile.info['sfreq']

In [13]:
amp.start()

In [14]:
fig=plt.figure(figsize=(20,12))  # plotting...
th=fig.suptitle('', fontsize=16)
ah1=fig.add_subplot(211)
ah2=fig.add_subplot(212)

plt.subplots_adjust(hspace=0.3)

ah1.set_title("Fp1", fontsize=16, weight='bold')
ah2.set_title("ICA component (eye blink)", fontsize=16, weight='bold')
ah1.set_xlabel("Time [s]", fontsize=15)
ah1.set_ylabel("Voltage [\u03BCV]", fontsize=15)
ah2.set_xlabel("Time [s]", fontsize=15)
ah2.set_ylabel("Voltage [\u03BCV]", fontsize=15)
ah1.tick_params(axis='both', labelsize=13)
ah2.tick_params(axis='both', labelsize=13)
ah1.xaxis.labelpad = 10
ah2.xaxis.labelpad = 10

l1, = ah1.plot(sx, sy1, color='b', label='Raw')
l2, = ah1.plot(sx, sy2, color='r', label='Corrected')
l3, = ah2.plot(sx, sy3)

ah1.legend(fontsize=14)

featurefig = plt.figure()
featureth=featurefig.suptitle('')
featureah=featurefig.add_subplot(111)
featurel1, = featureah.plot(featuresx, featuresy1)

# l=LoopState(); l.start()
markeroffset = 0  # needed to store all data in one big mat/vector
t0=time.time()
curTime=time.time()
st=''

x_data=0
while curTime - t0 < totalTime:  # l.get_state() != 'Stop':
   
    # keep track of time:
    curTime = time.time()
    
    # this is where you get the data
    data, marker = amp.get_data()
    
    if data.shape[0] > 0:  # this is crucial for remembering filter state.
        
        cnt = io.convert_mushu_data(data, marker, sfreq, ch_names)

        f_cnt, rt_zi_bp = proc.lfilter(cnt, rt_b_bp, rt_a_bp, zi=rt_zi_bp)  # real-time data preprocessing...        
        
        
        # plotting...
        
        eb_data = cnt.data[:,channel_to_plot]
        
        data_corrected = multi_dot([pca_comps.T, M, S, M_inv, pca_comps, cnt.data.T])
        
        sy1.extend(eb_data)  # to visualize/plot -- sy1 and sy2 are deque's
        sy2.extend(data_corrected[channel_to_plot,:])
        sy3.extend(eye_comp[x_data:x_data+len(eb_data)])
        sx.extend(data_length[x_data:x_data+len(eb_data)]/fs)
        
        l1.set_ydata(sy1)  
        l2.set_ydata(sy2)
        l3.set_ydata(sy3)
        l1.set_xdata(sx)
        l2.set_xdata(sx)
        l3.set_xdata(sx)
        
        msy1=np.mean(sy1)
        msy2=np.mean(sy2)
        msy3=np.mean(sy3)
         
        ah1.set_ylim(-150+msy1, 200+msy1)
        ah2.set_ylim(-50+msy3, 50+msy3)

        ah1.set_xlim(min(sx), max(sx))
        ah2.set_xlim(min(sx), max(sx))
        
        x_data += len(eb_data)
        
        fig.canvas.draw()
        fig.canvas.flush_events()
        
        # currently has no purpose
        newsamples = cnt.data.shape[0]

        # append to ringbuffer, so we can calculate features later on on the last N secs/samples of data.
        rb.append(f_cnt)

        # append it to the big matrix, for saving later on with pickle.
        alld.extend(data)
        for m in marker:
            allm.append([m[0] + markeroffset, m[1]])
        markeroffset += newsamples / float(sfreq) * 1000.
        


        # do the following every 0.1 msec - with with the ringbuffer:
        if curTime - markTime > updateTime:
            # do Stuff

            markTime = curTime
            # 1) obtain last 1-second(s)
            d = rb.get()

            # thomas does stuff here - in this example, we take channel 1 of the data and filter it
            feature = np.log10(np.mean(abs(d.data[:,0]))) * 10
            featuresy1.append(feature)
            
            # we send the value to BCI/STIM here - but not right now
            # bcinet.send_signal(bcixml.BciSignal({'nfsignal': signalToSend},None, bcixml.CONTROL_SIGNAL))
            featureMax = 0
            featureMin = -6.0
            featureScaling = 1/abs(featureMax - featureMin)
            featureOffset = (featureMax + featureMin) / 2
            signalToSend = featureScaling * (feature - featureOffset)

            # plot of the feature in a separate figure, to keep track:
            featurel1.set_ydata(featuresy1) # plotting the feature stuff
            featuremsy1=np.mean(featuresy1)
            featureah.set_ylim(-10+featuremsy1, 10+featuremsy1)
            
            featurefig.canvas.draw()
            featurefig.canvas.flush_events()

            # clear_output(wait=True)  # write some logging information here
            # clear_output clear the output of the cell, but if you do that you also remove the figures, it seems
            # so don't do it!
            str1 = '\nt = %f' % (curTime - t0)
            #str2 = 'Length Markers: %d' % len(allm)
            #str3 = '%d, %d' % data.shape
            str4 = 'Feature Value: %f' % feature
            str5 = 'Scaled Signal for NF: %f' % signalToSend
            #str6 = 'Eyeblinks: %d' % eyeblinks
            #print(str1 + '\n' + str2 + '\n' + str3 + '\n' + str4 + '\n' + str5)
            
            # print('Length Markers: %d' % len(allm))
            # print(data.shape)
            #th.set_text(str1 + '\n' + str2 + '\n' +str3 + '\n' + str6 + '\n')
            th.set_text(str1 + 's\n')
            featureth.set_text(str4 + '\n' + str5)
            

In [13]:
len(featuresy1)

25

ERROR! Session/line number was not unique in database. History logging moved to new session 623


In [14]:
amp.stop()
alld.shrink_to_fit()

amplifier stopped!


In [15]:
# write to disk, so we can re-load it later:

t={'alld':alld, 'allm':allm, 'ch_names':ch_names, 'sfreq':sfreq}
with open('c-allm-and-alld.pkl', 'wb') as f:
    pickle.dump(t, f)

In [16]:
# load from disk:

with open('c-allm-and-alld.pkl','rb') as f:
    t=pickle.load(f)
for key in t.keys():
    locals()[key] = t[key]

In [17]:
raw = convert_alld_allm_to_mne(alld, allm, ch_names, sfreq)  # covert to MNE
raw.resample(1000)

The following EEG sensors did not have a position specified in the selected montage: ['EOG']. Their position has been left untouched.


../../nftools\nftools\analysis.py:31: RuntimeWarning: The following EEG sensors did not have a position specified in the selected montage: ['EOG']. Their position has been left untouched.
  montage=montage


Creating RawArray with float64 data, n_channels=64, n_times=1169156
    Range : 0 ... 1169155 =      0.000 ...   584.577 secs
Ready.
2000.0
Creating RawArray with float64 data, n_channels=1, n_times=1169156
    Range : 0 ... 1169155 =      0.000 ...   584.577 secs
Ready.
1 events found
Event IDs: [1001]
1 events found
Event IDs: [1001]


<RawArray  |  None, n_channels x n_times : 65 x 584578 (584.6 sec), ~290.1 MB, data loaded>

In [18]:
raw.plot(scalings='auto');

In [19]:
# raw.set_eeg_reference(ref_channels='average')
# better not (yet) - before removing bad channels, since these mess up your data big time: see PREP paper:
%matplotlib qt5

In [27]:
picks = mne.pick_types(raw.info, meg=False, eeg=True, eog=False,
                       stim=False, exclude='bads')

raw.notch_filter(np.arange(50, 300, 50), picks=picks, filter_length='auto', phase='zero')
# add it (potentialy) some other preprocessing steps here

Setting up band-stop filter
Filter length of 6601 samples (6.601 sec) selected


<RawArray  |  None, n_channels x n_times : 65 x 584578 (584.6 sec), ~290.1 MB, data loaded>

In [28]:
# split data sets between EO and EC

newraw_eo=select_part_from_mne_dataset(raw, markers=[201, 202])
newraw_ec=select_part_from_mne_dataset(raw, markers=[203, 204])

using markers
1 events found
Event IDs: [1001]
using boundaries


IndexError: list index out of range

In [22]:
d1=plot_compare_two_spectra(newraw_eo, newraw_ec, freqs=[1, 25], n_fft=2048, n_overlap=512, chs_to_include=['Oz','O1'], freq_lims_topoplot=[7, 12], pow_lims = [-10, 25]);


NameError: name 'newraw_eo' is not defined

In [23]:
plt.figure();plt.plot(newraw_eo[0,0:1000][0].transpose());

NameError: name 'newraw_eo' is not defined

In [24]:
newraw_eo.plot(scalings='auto');

NameError: name 'newraw_eo' is not defined

In [25]:
mne.viz.plot_sensors(newraw_ec.info, show_names=True);

NameError: name 'newraw_ec' is not defined

In [26]:
newraw_eo.plot_psd(tmax=np.inf, fmax=250, n_fft=2048);
newraw_ec.plot_psd(tmax=np.inf, fmax=250, n_fft=2048);

Effective window size : 2.048 (s)


C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\mne\viz\evoked.py:162: RuntimeWarning: invalid value encountered in maximum
  rgb /= np.maximum(rgb.max(0), 1e-16)  # avoid div by zero
C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\matplotlib\colors.py:233: RuntimeWarning: invalid value encountered in less
  if np.any((result < 0) | (result > 1)):
C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\matplotlib\colors.py:233: RuntimeWarning: invalid value encountered in greater
  if np.any((result < 0) | (result > 1)):


Effective window size : 2.048 (s)


C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\mne\viz\evoked.py:162: RuntimeWarning: invalid value encountered in maximum
  rgb /= np.maximum(rgb.max(0), 1e-16)  # avoid div by zero
C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\matplotlib\colors.py:233: RuntimeWarning: invalid value encountered in less
  if np.any((result < 0) | (result > 1)):
C:\Users\thomasD\AppData\Local\Continuum\anaconda3\envs\rt\lib\site-packages\matplotlib\colors.py:233: RuntimeWarning: invalid value encountered in greater
  if np.any((result < 0) | (result > 1)):


In [27]:
## some attempts at doing ICA
#from mne.preprocessing import ICA
#method = 'fastica'
#n_components = 25
#decim = 3
#random_state = 23
#ica = ICA(n_components=n_components, method=method, random_state=random_state)
#print(ica)
#
#picks_eeg = mne.pick_types(newraw_ec.info, meg=False, eeg=True, eog=False,
#                           stim=False, exclude='bads')
#reject = dict(eeg=1e-3)
#ica.fit(newraw_ec, picks=picks_eeg, decim=decim, reject=reject)
#print(ica)

In [28]:
# close all of the figures.
%matplotlib qt5